In [11]:
#!pip install --upgrade pika

In [12]:
import pika
pika.__version__

'1.1.0'

In [13]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Setting permissions for user "myguest" in vhost "/" ...


In [14]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [15]:
!sudo rabbitmqctl list_bindings

Listing bindings ...


In [16]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic


In [17]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
exchangeName = 'direct_pings'

In [18]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

In [19]:
def on_request(ch, method, props, body):
    n = int(body) # body is the message to the worker. We want to calculate Fib(n)

    print(" I need to calculate fib(%s)" % n)
    
    print('ch:', ch)
    print('method:', method)
    print('props:', props)
    
    print('props.reply_to:', props.reply_to)
    print('props.correlation_id:', props.correlation_id)
    
    response = fib(n) # Results, Fib(n), calculated
    
    # We need to send back the results to the client
    ch.basic_publish(exchange='', # Default exchange
                     routing_key=props.reply_to, # This is the routing key to the queue that we use for sending the answer
                     #use the same correlation_id so the client knows that this is a reply to what message
                     properties=pika.BasicProperties(correlation_id = props.correlation_id), 
                     body=str(response))
    #ch.basic_ack(delivery_tag=method.delivery_tag)
    print(" Fib(%s) Calculation done. Results sent back to the server" % n)
    print(" *** \n")


In [20]:
channel.exchange_declare(exchange=exchangeName, exchange_type='direct')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

channel.queue_bind(exchange=exchangeName, queue=queue_name,routing_key='rpc_queue')

#channel.basic_qos(prefetch_count=1)


<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

In [21]:
#When the worker receives a message it will tun the "on_request" call back function
channel.basic_consume(queue=queue_name, on_message_callback=on_request, auto_ack=True)  

'ctag1.ac7489dad9f54f7897309f1185fed9c2'

In [ ]:
print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Awaiting RPC requests
 I need to calculate fib(30)
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7f078cecac50> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.ac7489dad9f54f7897309f1185fed9c2', 'delivery_tag=1', 'exchange=direct_pings', 'redelivered=False', 'routing_key=rpc_queue'])>
props: <BasicProperties(['content_type=text/plain', 'correlation_id=3276de27-7c7b-4f61-9aea-666aa4f7f211', 'delivery_mode=1', 'reply_to=amq.gen-JB1qmad_v6KbZQELdv9P0w'])>
props.reply_to: amq.gen-JB1qmad_v6KbZQELdv9P0w
props.correlation_id: 3276de27-7c7b-4f61-9aea-666aa4f7f211
 Fib(30) Calculation done. Results sent back to the server
 *** 

 I need to calculate fib(20)
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapt